In [72]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [73]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib, io, gzip, sys
from collections import defaultdict
np.random.seed(0)

In [74]:
df = pd.read_csv("/content/filter_matrix.csv", index_col=0)

In [75]:
df.head(5)

,NKAIN4,FREM2,FGF17,FGF9,PPP2R2B,TKTL1,C6orf15,BEX1,PAX2,FBXO2,...,BRINP1,EYA1,B3GAT1,INA,IGLON5,PGR,ADAM32,MMP10,SOX3,Sample_Type
TCGA-13-0924-01A,2.442390,5.002158,1.076459,6.861965,7.531925,0.000000,7.224835,3.456765,2.113221,8.532350,...,5.901332,3.595568,7.981338,3.946272,5.144637,6.573686,5.979473,1.076459,1.076459,Primary Tumor
TCGA-3P-A9WA-01A,10.987002,8.128193,4.627997,2.445926,8.847698,5.358916,0.000000,11.168978,6.173528,6.259815,...,3.842602,10.766483,9.298651,7.745104,12.769544,0.000000,5.642016,3.842602,2.445926,Primary Tumor
TCGA-61-2094-01A,5.374507,5.878609,4.341873,8.550752,12.135278,8.909991,7.185749,11.160721,8.601757,10.469605,...,8.291449,0.000000,5.729328,9.239394,2.762035,8.783155,6.936097,7.524698,0.000000,Primary Tumor
TCGA-59-2352-01A,2.918510,8.768812,5.617987,6.960685,4.027524,13.055713,6.651098,9.834361,8.520827,8.842828,...,0.000000,3.819801,6.502430,9.765346,9.194736,14.155738,6.882622,2.097721,12.009958,Primary Tumor
TCGA-09-0369-01A,3.971579,4.965311,2.817204,7.352535,4.750595,3.803216,6.824247,6.930253,3.971579,8.245066,...,2.817204,1.447836,4.192183,7.148028,4.442003,7.225032,9.400407,6.463742,0.000000,Primary Tumor


In [76]:
X = df.iloc[:, :-1]

In [77]:
y = df.iloc[:, -1]

In [78]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(x_train.shape, x_test.shape, y_test.shape, y_train.shape)

(12, 103) (4, 103) (4,) (12,)


# preprocessing

Handling missing values

In [80]:
(x_train == 0).sum().sum()
(x_train == 0).sum().sum() / (x_train.shape[0] * x_train.shape[1])*100

6.229773462783172

scaling

In [81]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

#fitting ssc on x_train
scaler_fit = scaler.fit(x_train)
#transform both train and test data
x_train_scaled = scaler_fit.transform(x_train)
x_test_scaled = scaler_fit.transform(x_test)

In [82]:
x_train.head(2)

,NKAIN4,FREM2,FGF17,FGF9,PPP2R2B,TKTL1,C6orf15,BEX1,PAX2,FBXO2,...,NLRP7,BRINP1,EYA1,B3GAT1,INA,IGLON5,PGR,ADAM32,MMP10,SOX3
TCGA-29-1766-01A,8.776084,4.170019,2.991867,9.126569,11.427936,2.282114,3.360837,6.008272,0.826014,10.384932,...,3.465471,8.662363,2.032497,6.869342,4.595946,7.177292,4.040508,8.283819,3.125619,2.282114
TCGA-13-1501-01A,2.834580,0.000000,2.347354,2.611439,7.397846,1.015944,1.606182,2.611439,2.023850,10.419707,...,10.443201,1.015944,0.000000,5.423284,4.117330,4.489719,7.832473,7.825988,4.117330,0.000000


# feature selection

In [83]:
# feature selection based on a univariate statistical test and selecting features based on a given significance level
from sklearn.feature_selection import SelectFpr

sfpr = SelectFpr(alpha=0.05)
sfpr_fit  = sfpr.fit(x_train ,  y_train)
x_train_sc_filt = sfpr_fit.transform(x_train)
x_test_sc_filt = sfpr_fit.transform(x_test)

In [ ]:
print(f"columns are reduced from {x_train.shape[1]} to {x_train_sc_filt.shape[1]}")

columns are reduced from 103 to 3


# building a model

In [86]:
#RF model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# rf= RandomForestClassifier()
# rf.fit(x_train_sc_filt,y_train)
# y_prep=rf.predict(x_test_sc_filt)
# a=accuracy_score(y_test, y_prep)
# a
model = RandomForestClassifier()
rf_fit=model.fit(x_train_sc_filt, y_train)
y_pred = model.predict(x_test_sc_filt)
rf_score = rf_fit.score(x_test_sc_filt, y_test)
print(round(rf_score,3))

0.75


In [87]:
y.value_counts()

,count
Sample_Type,
Primary Tumor,8
Recurrent Tumor,8
